# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../output_data/city.csv"
City_df = pd.read_csv(file)
#del City_df['Unnamed: 0']
City_df.shape

(581, 10)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = City_df[["Lat","Lng"]]
Humidity = City_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity,dissipating = False, max_intensity=10,point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df = City_df.loc[(City_df["Max Temp"] > 70 )& ( City_df["Max Temp"] < 80) &
                      (City_df["Wind Speed"] < 10) & (City_df["Cloudiness"]==0)]
#del hotel_df['Unnamed: 0']
hotel_df.dropna(inplace=True)
hotel_df

/Users/owner/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
41,41,Hong Kong,0,HK,1586456565,73,22.29,114.16,71.01,6.93
180,180,Pergamino,0,AR,1586456547,42,-33.89,-60.57,75.00,1.99
426,426,Porbandar,0,IN,1586456760,84,21.64,69.61,76.78,9.24
453,453,Presidencia Roque Sáenz Peña,0,AR,1586456768,28,-26.79,-60.44,77.14,5.10
454,454,Esna,0,EG,1586456768,33,25.29,32.55,77.00,3.36
511,511,Guaraniaçu,0,BR,1586456778,27,-25.10,-52.88,76.15,2.30
536,536,Neuquén,0,AR,1586456698,36,-38.95,-68.06,78.80,2.24
554,554,Ibirama,0,BR,1586456788,46,-27.06,-49.52,78.01,1.99
555,555,Māngrol,0,IN,1586456788,82,21.12,70.12,76.57,9.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

/Users/owner/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
41,41,Hong Kong,0,HK,1586456565,73,22.29,114.16,71.01,6.93,
180,180,Pergamino,0,AR,1586456547,42,-33.89,-60.57,75.00,1.99,
426,426,Porbandar,0,IN,1586456760,84,21.64,69.61,76.78,9.24,
453,453,Presidencia Roque Sáenz Peña,0,AR,1586456768,28,-26.79,-60.44,77.14,5.10,
454,454,Esna,0,EG,1586456768,33,25.29,32.55,77.00,3.36,


In [13]:
#target_coor = hotel_df.iloc[4,5],hotel_df.iloc[4,6]
target_search = "hotel"
target_radius = 5000
target_type = "hotel"


params = {
    #"location": target_coor,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

counter = 0
set = 1
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index,row in hotel_df.iterrows():
    if counter >= 50 :
        print("sleeping........")
        time.sleep(5)
        counter = 0
        set = set +1
    else:
        counter = counter + 1
    location = f"{row['Lat']},{row['Lng']}"
    
    params["location"] = location
    #print (params)
    
    response = requests.get(base_url, params=params)
    places_data = response.json()
    #print(json.dumps(places_data,indent=4,sort_keys=True))
    
    
    try:
        print(f"counter : {counter} , set : {set}, {places_data['results'][0]['name']}")
        hotel_df.loc[index,"Hotel Name"] = places_data["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    

# # print(json.dumps(places_data,indent=4,sort_keys=True))

hotel_df.head()

counter : 1 , set : 1, The Harbourview
counter : 2 , set : 1, Hotel Americano
counter : 3 , set : 1, Hotel Sheetal
counter : 4 , set : 1, Hotel Gualok
counter : 5 , set : 1, Dahabeya Queen Farida Sailing Boat - Esna/Aswan 5 Nights
counter : 6 , set : 1, Dalla´s Hotel
counter : 7 , set : 1, Neuquen Tower Hotel
counter : 8 , set : 1, Hotel Soralete
counter : 9 , set : 1, Kaveri Hotel


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
41,Hong Kong,0,HK,1586456565,73,22.29,114.16,71.01,6.93,The Harbourview
180,Pergamino,0,AR,1586456547,42,-33.89,-60.57,75.00,1.99,Hotel Americano
426,Porbandar,0,IN,1586456760,84,21.64,69.61,76.78,9.24,Hotel Sheetal
453,Presidencia Roque Sáenz Peña,0,AR,1586456768,28,-26.79,-60.44,77.14,5.10,Hotel Gualok
454,Esna,0,EG,1586456768,33,25.29,32.55,77.00,3.36,Dahabeya Queen Farida Sailing Boat - Esna/Aswa...


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
coordinates = list(zip(hotel_df["Lat"],hotel_df["Lng"]))

#len(coordinates)

markers = gmaps.marker_layer(coordinates,info_box_content=hotel_info)

fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add marker layer ontop of heat map


# Display Map